# **General Data Preparation**

In this notebook, we perform general data preparation steps required for the project. These include:

- Moving all necessary files to the appropriate directories  
- Defining the naming conventions for the dataset  
- Splitting the data into training and testing subsets  

These steps ensure the data is properly organized and ready for further processing and model development.


# Splitting data into training and test set
- The data will be split into training and test sets. The training set will consist of 90% of the data, while the test set will consist of the remaining 10%. The object classes will be distributed uniformly across both sets.

- Defined path: full_data/train, full_data/test

## **Importing required libraries and modules**

In [ ]:
%pip install ruamel.yaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 18.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import shutil
import yaml
import json
from sklearn.model_selection import train_test_split
from ruamel.yaml import YAML

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PROJECT_BASE = "/content/drive/MyDrive/MLDL/6D-Pose-Estimation"

# **Loading and Aggregating Image Filenames from Structured Subdirectories**

The `load_images` function creates a list of image filenames from selected folders. It defines valid folder names from "01" to "15", but skips "03" and "07".

For each valid folder, it builds the full path by combining `source_path`, the folder name, and `dest_fold`. Then, it lists and sorts all filenames inside that folder and adds them to the `image_labels` list.

The function returns a list where each element contains the sorted filenames from one folder. This list is ready for further image processing or labeling tasks.

The same procedure is performed for both **depth** and **RGB** images.

In [ ]:
def load_images(source_path, dest_fold):

  valid_folders = [f"{i:02d}" for i in range(1, 16) if f"{i:02d}" not in ["03", "07"]]

  image_labels = []

  for folder in valid_folders:

    images_path = os.path.join(source_path, folder, dest_fold)
    image_labels.append(sorted(os.listdir(images_path)))

  return image_labels


### **RGB**

In [ ]:
source_path = os.path.join(PROJECT_BASE, "data/raw/data")
image_labels = load_images(source_path, "rgb")

### **Depth**

In [ ]:
source_path = os.path.join(PROJECT_BASE, "data/raw/data")
image_depth_labels = load_images(source_path, "depth")

### The naming convention for images is: *classId_image_number*


## **Consolidate and relabel all images**

This step takes every image from its original class-specific folder, copies them into a single directory, and renames each file by prefixing it with its class ID. The result is a unified dataset where each filename carries its class label, making downstream processing and organization much simpler.

In [ ]:
def move_all_images(image_labels, destination_path):

  images_base_path = os.path.join(PROJECT_BASE, "data/raw/data")
  valid_folders = [f"{i:02d}" for i in range(1, 16) if f"{i:02d}" not in ["03", "07"]]
  class_map = {idx: cls  for idx, cls in enumerate(valid_folders)}

  for i, image_group in enumerate(image_labels):
    for label in image_group:
      image_path = os.path.join(images_base_path, class_map[i], "rgb", label)
      dst_path = os.path.join(destination_path, f"{class_map[i]}_{label}")
      shutil.copy2(image_path, dst_path)

### **RGB**

In [ ]:
destination_path_rgb = os.path.join(PROJECT_BASE, "data/full_data/images")
move_all_images(image_labels, destination_path_rgb)

### **Depth**

In [ ]:
destination_path_depth = os.path.join(PROJECT_BASE, "data/full_data/depth")
move_all_images(image_depth_labels, destination_path_depth)

## **Split Images into Training and Test Sets**

This step assigns a class label to each image based on its folder name. It then uses `train_test_split` from scikit-learn to randomly divide the images into 90% for training and 10% for testing, while keeping the same class proportions in both sets.

The same procedure is performed for both **depth** and **RGB** images.



In [ ]:
def split_into_train_test(input_path, input_files):

  valid_folders = [f"{i:02d}" for i in range(1, 16) if f"{i:02d}" not in ["03", "07"]]
  class_map = {idx: cls  for idx, cls in enumerate(valid_folders)}
  labels = [class_map[i] for i, img_group in enumerate(image_labels) for _ in img_group]

  X_train, X_test, y_train, y_test = train_test_split(
    all_images, labels, test_size=0.1, random_state=42, stratify=labels
  )

  return X_train, X_test

### **RGB**

In [ ]:
all_images = sorted(os.listdir(destination_path_rgb))
train_images, test_images = split_into_train_test(destination_path_rgb, all_images)

### **Depth**

In [ ]:
all_depth_images = sorted(os.listdir(destination_path_depth))
train_depth_images, test_depth_images = split_into_train_test(destination_path_depth, all_depth_images)

In [ ]:
len(train_images), len(test_images)

(14220, 1580)


## **Moving Test Images to a Separate Folder**


In [ ]:
def move_test_images(X_test, source_root, dest_root):

    os.makedirs(dest_root, exist_ok=True)

    for img_name in X_test:
        src_path = os.path.join(source_root, img_name)
        dst_path = os.path.join(dest_root, img_name)

        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
        else:
            print(f"Warning: {src_path} does not exist!")


### **RGB**

In [ ]:
input_path = os.path.join(PROJECT_BASE, "data/full_data/images")
dest_path = os.path.join(PROJECT_BASE, "data/full_data/test/images")

move_test_images(test_images, input_path, dest_path)

### **Depth**

In [ ]:
input_path = os.path.join(PROJECT_BASE, "data/full_data/depth")
dest_path = os.path.join(PROJECT_BASE, "data/full_data/test/depth")

move_test_images(test_depth_images, input_path, dest_path)

- After moving the test images, all remaining images in the `full_data/images` folder are considered part of the training set.


# **Generating the Appropriate Ground Truth Data**

In this step, we create a unified `gt.json` file that contains ground truth data from all class-specific `.yml` files, combined into a single structure.

After that, we perform the same procedure as with the images — we separate `gt.json` files for the training set and for the test set.


### **Formatting the YML File for Class 02**

The YML file for class `02` had a slightly different structure compared to the others. It contained ground truth data for all objects present in the image, not just for class `02`, as is the case with the other class-specific files.

In this step, we extract only the relevant ground truth information for class `02` to make the structure consistent with the rest of the dataset.


In [ ]:
yaml_loader = YAML()
yaml_loader.width = 1000
yaml_loader.indent(mapping=2, sequence=4, offset=2)
yaml_loader.default_flow_style = False

def force_flow_style_lists(obj):
  if isinstance(obj, list):
      seq = yaml_loader.seq(obj)
      seq.fa.set_flow_style()
      return seq
  elif isinstance(obj, dict):
      return {k: force_flow_style_lists(v) for k, v in obj.items()}
  else:
      return obj

def format_yml_file(input_path, output_path, object_id):

  with open(input_path, "r") as f:
      data = yaml_loader.load(f)

  filtered = {}
  for frame, objects in data.items():
      result = []
      for obj in objects:
          if obj.get("obj_id") == object_id:
              result.append(force_flow_style_lists(obj))
      if result:
          filtered[int(frame)] = result

  with open(output_path, "w") as f:
      yaml_loader.dump(filtered, f)


In [ ]:
labels_base_path = os.path.join(PROJECT_BASE, "data/raw/data")
yml_input_path = os.path.join(labels_base_path, "02", "gt.yml")
yml_output_path = os.path.join(labels_base_path, "02", "gt_new.yml") #later we renamed to the gt.yml
format_yml_file(yml_input_path, yml_output_path, 2)

### **Loading Ground Truth (.yml) Files for All Classes**


In [ ]:
def read_yml_file(path):
  with open(path, 'r') as file:
    data = yaml.safe_load(file)
  return data

In [ ]:
labels_base_path = os.path.join(PROJECT_BASE, "data/raw/data")
valid_folders = [f"{i:02d}" for i in range(1, 16) if f"{i:02d}" not in ["03", "07"]]

yml_files = []

for folder in valid_folders:

  yml_path = os.path.join(labels_base_path, folder, "gt.yml")
  yml_files.append(read_yml_file(yml_path))

### **Merging All YML Files into a Single JSON File**

In [ ]:
def make_integrated_json_file(yml_files, class_map, json_output_path):

  gt_rot_trans_bb_data = {}

  for i, yml in enumerate(yml_files):

    for key, value in yml.items():

      rotation = value[0]['cam_R_m2c']
      translation = value[0]['cam_t_m2c']
      border_box = value[0]['obj_bb']

      gt_rot_trans_bb_data[f"{class_map[i]}_{key:04d}"] = [rotation, translation, border_box]

  # Exporting JSON file
  with open(json_output_path, 'w') as f:
    json.dump(gt_rot_trans_bb_data, f, indent=2)

  return gt_rot_trans_bb_data

In [ ]:
json_output_path = os.path.join(PROJECT_BASE, "data", "full_data", "gt.json")
class_map = {idx: cls  for idx, cls in enumerate(valid_folders)}
gt_rot_trans_bb_data = make_integrated_json_file(yml_files, class_map, json_output_path)

### **Splitting integrated json for training and test**

In [ ]:
def split_json(json_path, train_images, test_images, output_path_train, output_path_test):

  #Load json
  with open(json_path, "r") as f:
    data = json.load(f)
  train_json = {key:value for key, value in data.items() if f"{key}.png" in train_images}
  test_json = {key:value for key, value in data.items() if f"{key}.png" in test_images}

    # Export JSONs
  with open(output_path_train, "w") as f:
      json.dump(train_json, f, indent=4)

  with open(output_path_test, "w") as f:
      json.dump(test_json, f, indent=4)

In [ ]:
train_images = os.listdir(os.path.join(PROJECT_BASE, "data/full_data/train/images"))
test_images = os.listdir(os.path.join(PROJECT_BASE, "data/full_data/test/images"))

output_train_json = os.path.join(PROJECT_BASE, "data/full_data/train/gt.json")
output_test_json = os.path.join(PROJECT_BASE, "data/full_data/test/gt.json")

split_json(json_output_path, train_images, test_images, output_train_json, output_test_json)

# **All further experiments and additional data preparation in other notebooks will be performed only on the training set.**
